In [1]:
# Write down all paths.


PATH1 = r"C:\Users\enesy\Videos\xd\1.mp4"
PATH2 = r"C:\Users\enesy\Videos\xd\2.MOV"
PATH3 = r"C:\Users\enesy\Videos\xd\3.mp4"
PATH4 = r"C:\Users\enesy\Videos\xd\4.MP4"

In [2]:
# Find all delays
# Credit: Bucky und Stef

from prototype import *

all_paths = [PATH1, PATH2, PATH3, PATH4]

delays = find_all_delays(all_paths)

delays

pygame 2.6.1 (SDL 2.28.4, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'avc1', 'minor_version': '0', 'compatible_brands': 'avc1isom', 'creation_time': '2024-11-16T09:45:49.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': 'eng', 'default': True, 'size': [1920, 1080], 'bitrate': 23988, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'rotate': 180.0, 'creation_time': '2024-11-16T09:45:49.000000Z', 'handler_name': 'Ambarella AVC', 'encoder': 'Ambarella AVC encoder', 'Side data': '', 'displaymatrix': 'rotation of -180.00 degrees'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 192, 'metadata': {'rotate': '180', 'creation_time': '2024-11-16T09:45:49.000000Z', 'handler_name': 'Ambarella AAC'}}], 'input_number': 0}], 'dura

[0.0, 18.829319727891157, 0.3985487528344671, 12.75968253968254]

In [5]:
# We need to know the durations of videos
# We will update these as we go.

from moviepy import VideoFileClip

durations = [VideoFileClip(path).duration for path in all_paths]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'avc1', 'minor_version': '0', 'compatible_brands': 'avc1isom', 'creation_time': '2024-11-16T09:45:49.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': 'eng', 'default': True, 'size': [1920, 1080], 'bitrate': 23988, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'rotate': 180.0, 'creation_time': '2024-11-16T09:45:49.000000Z', 'handler_name': 'Ambarella AVC', 'encoder': 'Ambarella AVC encoder', 'Side data': '', 'displaymatrix': 'rotation of -180.00 degrees'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 192, 'metadata': {'rotate': '180', 'creation_time': '2024-11-16T09:45:49.000000Z', 'handler_name': 'Ambarella AAC'}}], 'input_number': 0}], 'duration': 415.72, 'bitrate': 24234, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0,

In [21]:
import ffmpeg

# Load videos into ffmpeg
videos = [ffmpeg.input(path) for path in all_paths]

# We handle each video individually.
for index, video in enumerate(videos):

    # If the delay of the vide is negative:
    if delays[index] < 0:
        # We trim the video from the start
        videos[index] = ffmpeg.trim(video, start=round(delays[index])).setpts("PTS-STARTPTS")
        
        # Update the corresponding video's duration
        durations[index] -= round(delays[index])

    else: # If the delay is positive
        # We delay/push the videos start by the amount

        videos[index] = ffmpeg.filter(video, "tpad", start_duration=round(delays[index], 2))
        
# Put the videos on grid 2x2
xstacked = ffmpeg.filter(videos, "xstack", inputs="4", layout="0_0|0_h0|w0_0|w0_h0")

# We cut the whole video by the longest positive delay
# So that the videos begin at the same time.
start_end_adjusted = ffmpeg.trim(xstacked, start=max(delays)).setpts("PTS-STARTPTS")

# Scale down the video to 1080p 60FPS
# Normally, we are supposed to output 4K 60FPS
scaled = ffmpeg.filter(start_end_adjusted, "scale", 1920, -1).filter('fps', fps=60, round='up')

# The output uses ultrafast preset for demonstration purposes.
# The video will continue until the shortest video ends, i.e set stop_duration = min(durations)
# If you want to test it out for a shorter time, change the "stop_duration" to amount of seconds you want (e.g 10 secs)
stop_duration = min(durations)
out = ffmpeg.output(scaled, "xd.mp4", vcodec="libx264", crf=27, preset="ultrafast", progress="pipe:1", to=stop_duration)

# Compile command will return the terminal command to run FFMPEG as a string
command = out.compile()

In [23]:
# We are going to use pipes and subprocess
# This is because we want to see the progress of FFMPEG from our main process (i.e the notebook)

import subprocess
import datetime # For printing progress

# Run the FFMPEG subprocess
try:
    ffmpeg_process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
except OSError:
    print("OSError")
except ValueError:
    print("ValueError: Cannot run FFMPEG with given parameters.")

end_time = str(datetime.timedelta(seconds=stop_duration)) # Used for printing the last point of the video
# Print the progress that ffmpeg outputs
print("FFMPEG is starting, please wait a couple minutes.")
for line in ffmpeg_process.stdout:
    # print(line)

    if line.startswith("out_time="):
        progress = line.split("=")[1]
        print(f"Progress: currently at {progress} / going until {end_time}", end="\r")
print("FFMPEG has finished processing.")

# IMPORTANT: you have to quit the FFMPEG process itself from Task Manager to stop it
# It will also not continue if the file exists already, make sure to delete the file if it looks like its doing nothing.


FFMPEG is starting, please wait a couple minutes.
Progress: currently at N/A
Progress: currently at 00:00:00.050000
Progress: currently at 00:00:00.183333
Progress: currently at 00:00:00.250000
Progress: currently at 00:00:00.350000
Progress: currently at 00:00:00.383333
Progress: currently at 00:00:00.516667
Progress: currently at 00:00:00.683333
Progress: currently at 00:00:00.833333
Progress: currently at 00:00:00.916667
Progress: currently at 00:00:01.000000
Progress: currently at 00:00:01.116667
Progress: currently at 00:00:01.150000
Progress: currently at 00:00:01.233333
Progress: currently at 00:00:01.383333
Progress: currently at 00:00:01.466667
Progress: currently at 00:00:01.633333
Progress: currently at 00:00:01.716667
Progress: currently at 00:00:01.833333
Progress: currently at 00:00:01.916667
Progress: currently at 00:00:02.033333
Progress: currently at 00:00:02.133333
Progress: currently at 00:00:02.216667
Progress: currently at 00:00:02.316667
Progress: currently at 00: